In [1]:
#!pip install celery

In [2]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

In [3]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


This code will **run on a server machine**. It will ask worker machines to complete some tasks and send the results back to the server.

In the same folder with this code there is a test.py file 
You will need to run that code on a **worker** machine with <br>

**"celery -A test worker --loglevel=info"**

Then that machine will become a worker, and will be able to run the app task, i.e. in this case the echo function, whenever the broker requests it.

The echo function will be run by these commands on the remote (=worker) machine:

from test import echo
res = echo.delay('Python rocks!'); print(type(res)); print(res)
res.result

In [4]:
from test import echo

Before you run the next cell, you will need to <br>
"pip install celery" on the worker machine and then run <br>
"celery -A test worker --loglevel=info" on the worker machine.

Otherwise the result of 'res1.ready()' in the cell below will be False

In [5]:
# We will run the echo function on the worker machine.
# The fun.delay(arg) function sends the arg message to the worker's fun function and run it on the worker machine.
# For more info, see https://docs.celeryproject.org/en/latest/userguide/calling.html
    
# This code will send 'Python rocks 1 :) !' to the worker machine, and put its response into res1
res1 = echo.delay('Python rocks 1 :) !'); print(type(res1)); print(res1)

<class 'celery.result.AsyncResult'>
af36a5cf-8b45-448b-8915-d4d86849f0ef


In [6]:
res1.ready()

True

In [7]:
res1.result

'Response from worker: Python rocks 1 :) !'

In [8]:
# Let us repeat it with a different message: 
res2 = echo.delay('Python rocks 2 :) !'); print(type(res2)); print(res2)
res2.ready()

<class 'celery.result.AsyncResult'>
b3b8079b-e8de-4260-8afd-dfc86f3f102c


In [9]:
print(res2.result)

Response from worker: Python rocks 2 :) !
